In [34]:
import os
import re
import numpy as np
import tifffile as tiff
import cv2

In [35]:
def max_projection_montage(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)  # Ensure output folder exists
    
    # Regex to match files containing '-F4' to '-F11'
    pattern = re.compile(r'-F([4-9]|10|11)')
    
    for filename in os.listdir(input_folder):
        if filename.endswith(".tif") and pattern.search(filename):
            input_path = os.path.join(input_folder, filename)
            
            # Read multi-dimensional TIF file with metadata
            with tiff.TiffFile(input_path) as tif_file:
                img = tif_file.asarray()
                metadata = tif_file.pages[0].tags
                resolution_tag = metadata.get('XResolution')  # Get resolution if available
                pixel_size = 0.1883734  # Default pixel size in microns
                if resolution_tag:
                    resolution = resolution_tag.value  # Extract actual value
                    pixel_size = resolution[1] / resolution[0]  # Convert to microns per pixel
            
            print(f"Processing {filename} with shape {img.shape}")
            
            # Ensure image has correct dimensions (Z, C, H, W) or (C, Z, H, W)
            if len(img.shape) == 4:
                if img.shape[1] == 4:  # (Z, C, H, W) format
                    img = np.moveaxis(img, 1, 0)  # Convert to (C, Z, H, W)
                
                # Now img is (C, Z, H, W)
                channels = [np.max(img[i, :, :, :], axis=0) for i in range(4)]
                
                # Debug: Print max intensity per channel
                for i, ch in enumerate(channels):
                    print(f"Channel {i} max intensity: {np.max(ch)}")
                
                # Find the brightest object in channel 3 (index 2)
                ch3 = channels[2]
                min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(ch3)
                center_x, center_y = max_loc
                print(f"Channel 3 max value: {max_val} at {max_loc}")
                
                if max_val == 0:
                    print(f"Skipping {filename}: No bright object found in channel 3.")
                    continue
                
                # Define cropping bounds (400x400 pixels around the brightest object)
                half_size = 200
                h, w = ch3.shape
                x1, x2 = center_x - half_size, center_x + half_size
                y1, y2 = center_y - half_size, center_y + half_size
                
                # Ensure crop remains within image bounds and pad if necessary
                pad_x1, pad_x2, pad_y1, pad_y2 = 0, 0, 0, 0
                if x1 < 0:
                    pad_x1 = abs(x1)
                    x1 = 0
                if x2 > w:
                    pad_x2 = x2 - w
                    x2 = w
                if y1 < 0:
                    pad_y1 = abs(y1)
                    y1 = 0
                if y2 > h:
                    pad_y2 = y2 - h
                    y2 = h
                
                print(f"Cropping coordinates: x1={x1}, x2={x2}, y1={y1}, y2={y2}")
                print(f"Padding applied: Left={pad_x1}, Right={pad_x2}, Top={pad_y1}, Bottom={pad_y2}")
                
                # Crop and pad each channel at the same position
                cropped_channels = [ch[y1:y2, x1:x2] for ch in channels]
                padded_channels = [
                    cv2.copyMakeBorder(ch, pad_y1, pad_y2, pad_x1, pad_x2, cv2.BORDER_CONSTANT, value=0)
                    for ch in cropped_channels
                ]
                
                # Normalize each channel to grayscale (0-255)
                def safe_normalize(img):
                    if np.max(img) == np.min(img):  # Avoid division by zero
                        return np.zeros_like(img, dtype=np.uint8)
                    return cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
                
                grayscale_channels = [safe_normalize(ch) for ch in padded_channels]
                
                # Arrange in a 2x2 montage
                top_row = np.hstack((grayscale_channels[0], grayscale_channels[1]))
                bottom_row = np.hstack((grayscale_channels[2], grayscale_channels[3]))
                montage = np.vstack((top_row, bottom_row))
                
                # Preserve metadata and pixel calibration
                with tiff.TiffWriter(os.path.join(output_folder, filename)) as tif:
                    tif.write(montage, metadata = {'axes': 'YX'}, resolution = (1.0 / pixel_size, 1.0 / pixel_size))
                
                print(f'Processed: {filename}')
            else:
                print(f'Skipping {filename}: Unexpected image dimensions {img.shape}')
    print("Done.")

In [36]:
input_folder = "/Volumes/arxivBeta/_Tobias/Opera/20250204/Importer_cases"  # Change this to your actual folder path
output_folder = "/Volumes/arxivBeta/_Tobias/Opera/20250204/Montages_cases_V2"  # Change this to your actual output folder path
#input_folder = "/Volumes/arxivBeta/_Tobias/Opera/20250204/In"  # Change this to your actual folder path
#output_folder = "/Volumes/arxivBeta/_Tobias/Opera/20250204/Out"  # Change this to your actual output folder path

max_projection_montage(input_folder, output_folder)

Processing ID0080_TK_CID_RNAiScreen_63x_ReScan_72h - R02-C07-F7_B7_CM_CENPJ_init.tif with shape (16, 4, 1080, 1080)
Channel 0 max intensity: 0
Channel 1 max intensity: 0
Channel 2 max intensity: 0
Channel 3 max intensity: 0
Channel 3 max value: 0.0 at (0, 0)
Skipping ID0080_TK_CID_RNAiScreen_63x_ReScan_72h - R02-C07-F7_B7_CM_CENPJ_init.tif: No bright object found in channel 3.
Processing ID0080_TK_CID_RNAiScreen_63x_ReScan_72h - R02-C07-F8_B7_CM_CENPJ_init.tif with shape (16, 4, 1080, 1080)
Channel 0 max intensity: 0
Channel 1 max intensity: 0
Channel 2 max intensity: 0
Channel 3 max intensity: 0
Channel 3 max value: 0.0 at (0, 0)
Skipping ID0080_TK_CID_RNAiScreen_63x_ReScan_72h - R02-C07-F8_B7_CM_CENPJ_init.tif: No bright object found in channel 3.
Processing ID0080_TK_CID_RNAiScreen_63x_ReScan_72h - R02-C07-F9_B7_CM_CENPJ_init.tif with shape (16, 4, 1080, 1080)
Channel 0 max intensity: 0
Channel 1 max intensity: 0
Channel 2 max intensity: 0
Channel 3 max intensity: 0
Channel 3 max va